In [ ]:
!pip install -U dartmouth-langchain > /dev/null
from dartmouth_langchain.llms import ChatDartmouth, DartmouthLLM
from langchain_core.messages import HumanMessage, SystemMessage
import pandas as pd
import numpy as np
import os

In [ ]:
def format_prompt(prompt):
    """
    This function will format a prompt into what is needed for LangChain to produce ChatML.
    Args:
       prompt: the text to be embedded as human prompt.
    """
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=prompt),
    ]
    return messages

In [ ]:
os.environ["DARTMOUTH_API_KEY"] = "KEY_GOES_HERE"

In [ ]:
# Less stochasticity, more reliable output
max_new_tokens = 1024
top_p = 0.95
temperature = .55
system_prompt = "Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity."
kwargs = dict()

llm_chat = ChatDartmouth(model_name="llama-3-1-8b-instruct",
                    temperature = temperature,
                    top_p = top_p,
                    max_tokens = max_new_tokens,
                    model_kwargs = kwargs)

In [ ]:
# Example: Prompting for completion
prompt = """Instructions: please complete the following text. Use only what you have seen in your training data.

The woods are lovely, dark and deep,   
But I have promises to keep,
"""

In [ ]:
print(llm_chat.invoke(format_prompt(prompt)).content)

In [ ]:
# Example: Masked Prompting for completion
prompt = """Instructions: In the following replace [MASK] with the correct word. Use only a single word.
Make sure to use only the words found from samples in your training data. You must make a guess, 
even if you are uncertain.

Example: 

Input: Whenever Richard [MASK] went down town,
Output <token>Cory</token>

Input: But a caged [MASK] stands on the grave of dreams
Output:"""

In [ ]:
print(llm_chat.invoke(format_prompt(prompt)).content)

In [ ]:
# Example: Accessing Logprobs for next token prediction

In [ ]:
prompt = """What is the next token. Provide just a single word. 
Dartmouth College's mission is to educate promising students and prepare them for a 
lifetime of learning and responsible"""

In [ ]:
# modify params to include logprobs
llm_chat = ChatDartmouth(model_name="llama-3-1-8b-instruct",
                    temperature = 1.5,
                    top_p = top_p,
                    logprobs = True,
                    top_logprobs = 5,
                    max_tokens = max_new_tokens,
                    model_kwargs = kwargs)

In [ ]:
# assign output to a variable
output = llm_chat.invoke(format_prompt(prompt))

In [ ]:
# gain access to logprobs
logprobs = output.response_metadata['logprobs']['content']

In [ ]:
# create a DataFrame of the logprobs, padding if necessary
rows = list()
for token in logprobs[:10]:
    tks = []
    for t in token['top_logprobs']:
        tks.append(t['token'] + " (" + str(np.round(t['logprob'],3)) + ")")
    tp = len(token['top_logprobs'])
    pad = 5 - tp
    for i in range(pad):
        tks.append("NA")
    rows.append(tks)
df = pd.DataFrame(rows,columns=["Selected","Tk1","Tk2","Tk3","Tk4"])

In [ ]:
# Display
df